# Run Deeplabut for freely moving box

We will first import the libraries needed. If you get an error, check that you're running the notebook inside the environment where DeepLabCut is installed.

In [1]:
import deeplabcut
import os, glob, re, datetime, shutil
import dlc_fun

# we will set some general path identifiers here
datapath       = 'S:\\ElboustaniLab\\#SHARE\\Data'
jointmod       = '0Dyad_JointPerceptualDecisionMaking'
filmidentifier = 'Filming'

Loading DLC 2.3.10...


We will now specify the model for analysis. Only one model is available, for unimplanted mice.

In [2]:
projectpath = 'S:\\ElboustaniLab\\#SHARE\\Analysis\\JointDecisionDeepLabCut\\NeuroPixelsBox-DK-2023-08-14'
config      = os.path.join(projectpath,'config.yaml')

### Find videos

Specify the mouse or pair for which you want to perform the analysis. This string should match folder namings.

In [10]:
mousefolders = ['YX001', 'YX002','YX003','YX004','YX005','YX006', 'YX007','YX008','YX009','YX010']
#mousefolders = ['YX001', 'YX002','YX003','YX004','YX005','YX006', 'YX007','YX008','YX009','YX010']
datestart    = '20220201'

In [19]:
mousepaths = []
for imouse in range(len(mousefolders)):
    if '_' in mousefolders[imouse]:
        mousepaths.append(os.path.join(datapath, jointmod, mousefolders[imouse]))
    else:
        mousepaths.append(os.path.join(datapath, mousefolders[imouse]))
        
for impath in range(len(mousepaths)):
    if os.path.isdir(mousepaths[impath]): 
        print(str(impath+1) + '. data path found: ' + mousepaths[impath])
    else:
        print(str(impath+1) + 'no such mouse or pair, check your list')


1. data path found: S:\ElboustaniLab\#SHARE\Data\YX001
2. data path found: S:\ElboustaniLab\#SHARE\Data\YX002
3. data path found: S:\ElboustaniLab\#SHARE\Data\YX003
4. data path found: S:\ElboustaniLab\#SHARE\Data\YX004
5. data path found: S:\ElboustaniLab\#SHARE\Data\YX005
6. data path found: S:\ElboustaniLab\#SHARE\Data\YX006
7. data path found: S:\ElboustaniLab\#SHARE\Data\YX007
8. data path found: S:\ElboustaniLab\#SHARE\Data\YX008
9. data path found: S:\ElboustaniLab\#SHARE\Data\YX009
10. data path found: S:\ElboustaniLab\#SHARE\Data\YX010


In [20]:
listvidpaths = []
for imfold in range(len(mousepaths)):
    allfpaths = dlc_fun.find_mp4_files(mousepaths[imfold])
    listvidpaths.extend(allfpaths)

In [21]:
# Create a list of tuples (date, path)
dated_paths = [(dlc_fun.extract_date(path), path) for path in listvidpaths]

# Filter out None dates and sort the list by date
sorted_paths = sorted((dp for dp in dated_paths if dp[0] is not None), key=lambda x: x[0])

datethres    = datetime.datetime.strptime(datestart, '%Y%m%d')

# Extract the sorted paths
sorted_paths_only = [path for date, path in sorted_paths 
                     if datetime.datetime.strptime(date, '%Y%m%d') >= datethres and 
                     ('Unsorted' not in path) and ('Habituation' not in path) and 
                     ('Observational' not in path) and  ('ToSort' not in path) and  ('Direction' not in path)]
 
# paths analyzed
pathsold = [path for path in sorted_paths_only if not dlc_fun.to_analyze_dlc(path)]

# clean up pickle and h5 pathsold
for ipath in range(len(pathsold)):
    dlc_fun.remove_big_files(os.path.splitext(pathsold[ipath])[0])
        
# paths to run DLC for
pathsrun = [path for path in sorted_paths_only if dlc_fun.to_analyze_dlc(path)]
pathsrun.reverse()

if (len(sorted_paths_only)==0):
    print('No video folder found.')
else:
    print('The following files were found that have not been analyzed:')
    for ivid in range(len(pathsrun)):
        print(pathsrun[ivid])
       

The following files were found that have not been analyzed:
S:\ElboustaniLab\#SHARE\Data\YX009\Filming\OrientationSingleMouse\20241120\Session1\20241120_1436_YX009_1_reduced.mp4
S:\ElboustaniLab\#SHARE\Data\YX008\Filming\OrientationSingleMouse\20241120\Session1\20241120_1319_YX008_1.mp4
S:\ElboustaniLab\#SHARE\Data\YX007\Filming\OrientationSingleMouse\20241120\Session1\20241120_1315_YX007_1.mp4
S:\ElboustaniLab\#SHARE\Data\YX006\Filming\OrientationSingleMouse\20241120\Session1\20241120_1312_YX006_1.mp4
S:\ElboustaniLab\#SHARE\Data\YX002\Filming\Tests\20241120\Session1\20241120_1441_YX002_1.mp4
S:\ElboustaniLab\#SHARE\Data\YX010\Filming\OrientationSingleMouse\20241119\Session1\20241119_1347_YX010_1_reduced.mp4
S:\ElboustaniLab\#SHARE\Data\YX009\Filming\OrientationSingleMouse\20241119\Session1\20241119_1353_YX009_1_reduced.mp4
S:\ElboustaniLab\#SHARE\Data\YX008\Filming\OrientationSingleMouse\20241119\Session1\20241119_1234_YX008_1.mp4
S:\ElboustaniLab\#SHARE\Data\YX007\Filming\Orientatio

### Run model

Let's run deeplabcut now.

In [ ]:
for ipath in range(len(pathsrun)):
    
    currpath = pathsrun[ipath]
    # find if we can crop video
    ispair   = jointmod in currpath
    skipcrop = ispair or ('Visual' in currpath)

    localpath = dlc_fun.copy_video_locally(currpath, 'C:\\Temp_proc')

    # do cropping
    if skipcrop:
        cropval = None
    else:
        cropval = dlc_fun.get_mouse_compartment(localpath)
        
    # run DLC locally
    deeplabcut.analyze_videos(config, localpath, videotype = 'mp4', cropping = cropval,
                              auto_track = True, n_tracks = 2, save_as_csv = True, batchsize = 4)
    
    csvpath = glob.glob(os.path.splitext(localpath)[0] + '*_el.csv')
    
    # if video was cropped, fix csv file
    if not skipcrop:
        dlc_fun.update_saved_csv(csvpath[0], cropval[2])

    # move dlc csv back to path
    shutil.copy2(csvpath[0], os.path.split(currpath)[0])
    
    # remove h5 and pickles after you're done, and other local files
    dlc_fun.remove_big_files(os.path.splitext(localpath)[0])
    os.remove(localpath)
    os.remove(csvpath[0])

Copying file to C:\Temp_proc\20241120_1436_YX009_1_reduced.mp4
determining mouse side...
frames done:


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [01:59<00:00,  3.36it/s]

1034.8297204177663
139.66017572691368
mouse found at the TOP part of the video


Overwriting cropping parameters: [0, 1408, 0, 804]
These are used for all videos, but won't be save to the cfg file.
Using snapshot-100000 for model S:\ElboustaniLab\#SHARE\Analysis\JointDecisionDeepLabCut\NeuroPixelsBox-DK-2023-08-14\dlc-models\iteration-3\NeuroPixelsBoxAug14-trainset95shuffle1


C:\Users\usr_m_elbousta1\AppData\Local\anaconda3\envs\DEEPLABCUT\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


...

1011.1729290070375
154.36362871924445
mouse found at the TOP part of the video


Overwriting cropping parameters: [0, 1408, 0, 804]
These are used for all videos, but won't be save to the cfg file.
Using snapshot-100000 for model S:\ElboustaniLab\#SHARE\Analysis\JointDecisionDeepLabCut\NeuroPixelsBox-DK-2023-08-14\dlc-models\iteration-3\NeuroPixelsBoxAug14-trainset95shuffle1


C:\Users\usr_m_elbousta1\AppData\Local\anaconda3\envs\DEEPLABCUT\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Activating extracting of PAFs
Starting to analyze %  C:\Temp_proc\20241120_1312_YX006_1.mp4
Loading  C:\Temp_proc\20241120_1312_YX006_1.mp4
Duration of video [s]:  19277.23 , recorded with  30.0 fps!
Overall # of frames:  578317  found with (before cropping) frame dimensions:  1408 1408
Starting to extract posture from the video(s) with batchsize: 4


  1%|▍                                                                         | 3400/578317 [01:47<5:02:19, 31.69it/s]